In [ ]:
!pip install estnltk==1.7.2

In [ ]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
tqdm.pandas()

# for morphological analysis
from estnltk import Text
from estnltk.taggers import NerTagger
from estnltk.taggers import SentenceFleschScoreRetagger
from estnltk.converters.conll import conll_exporter

# for dependency graph
from nltk.parse import DependencyGraph
from graphviz import Source

In [ ]:
# google drive
from google.colab import drive
from google.colab import files

drive.mount("/content/drive")
data = pd.read_csv("/content/drive/MyDrive/EstonianStanceDetection/notebooks/data.csv")

In [ ]:
# local
data = pd.read_csv("data.csv")

In [ ]:
data.head(50)

,sentence,stanceConsolidated,stance,publishDate,title,publisherGroup,publisher,category,keywordGroups,keywords,author,morph_analysis,sentence_count,word_count,flesch_score,dependency_tree_height,named_entities,named_entities_count
0,"Ja selliseid inimesi on Eesti võimukoridorides ohtralt! Veel hullem on kogu see maailma päästmine, kus meile on Brüsseli käsul oma huvidest tähtsamad Süüria sõjapõgenikud, üle Vahemere tulevad neegritest majandusimmigrandid, Lääne-Euroopas välja mõeldud geiõigused ja sooneutraalsus, mujalt siia tulevad töölised ja nii edasi – ainult mitte omad inimesed.",1,against,29.05.2018 08:41,"Eesti riigivõim ei ehita oma riiki, vaid tegeleb Suure Hundi käsul maailma parandamisega",UuedUudised,UuedUudised,Arvamus,"['ränne', 'pagulased', 'rass']","['migra', 'põgenik', '([\\W ]|^)neeg']",NaN,"Text(text='Ja selliseid inimesi on Eesti võimukoridorides ohtralt! Veel hullem on kogu see maailma päästmine, kus meile on Brüsseli käsul oma huvidest tähtsamad Süüria sõjapõgenikud, üle Vahemere tulevad neegritest majandusimmigrandid, Lääne-Euroopas välja mõeldud geiõigused ja sooneutraalsus, mujalt siia tulevad töölised ja nii edasi – ainult mitte omad inimesed.')",2,23.0,52.337955,7.0,"['Eesti', 'Brüsseli', 'Süüria', 'Vahemere', 'Lääne-Euroopas']",2.5
1,Massiimmigratsioon oleks Euroopale hukatuslik ja see ei lahendaks maailmas mitte midagi.,2,against,13.10.2020 08:31,Aafriklaste massiränne Euroopasse tähendab Musta Mandri kolimist külmemasse kliimasse ja muud ei midagi,UuedUudised,UuedUudised,Maailm,['ränne'],['migra'],NaN,Text(text='Massiimmigratsioon oleks Euroopale hukatuslik ja see ei lahendaks maailmas mitte midagi.'),1,11.0,73.142174,4.0,['Euroopale'],1.0
2,"Lõppjäreldus on, et Eesti rahvaarv kasvab võõraste sisserändest, mis on rahvusriigile ohtlik.“Iga-aastase paarituhandelise sisserände ülekaalu puhul hakkaks eestlaste suhtarv rahvastikus aegamööda vähenema.”",2,against,21.01.2019 10:22,"Eesti rahvaarv suureneb, kuid rahvusriigina pole põhjust hõisata",UuedUudised,UuedUudised,Eesti,['ränne'],['rände'],NaN,"Text(text='Lõppjäreldus on, et Eesti rahvaarv kasvab võõraste sisserändest, mis on rahvusriigile ohtlik.“Iga-aastase paarituhandelise sisserände ülekaalu puhul hakkaks eestlaste suhtarv rahvastikus aegamööda vähenema.”')",2,11.5,64.933462,5.0,['Eesti'],0.5
3,"Kui meeleavaldajad hakkavad liikuma, organiseerijate palvel kõhedusttekitavalt vaikides, ühineb pensionär nendega.ee juhtus demonstratsioonil, mille korraldas PEGIDA ehk Patriotische Europäer gegen die Islamisierung des Abendlandes (patriootlikud eurooplased õhtumaa islamiseerimise vastu)",3,neutral,06.01.2015 01:30,Dresdeni rahvahulgad peegeldavad Euroopa hirmu migrantide ees,EkspressGrupp,Eesti Päevaleht,NaN,['rahvus'],['islam'],NaN,"Text(text='Kui meeleavaldajad hakkavad liikuma, organiseerijate palvel kõhedusttekitavalt vaikides, ühineb pensionär nendega.ee juhtus demonstratsioonil, mille korraldas PEGIDA ehk Patriotische Europäer gegen die Islamisierung des Abendlandes (patriootlikud eurooplased õhtumaa islamiseerimise vastu)')",1,29.0,8.604286,8.0,"['PEGIDA', 'Patriotische Europäer', 'Islamisierung', 'Abendlandes']",4.0
4,"See sõda tähendab, et peame ka Eestis oma julgeolekus arvesse võtma islami ekstremismi ohu hoopis uut olukorda silmas pidades.",3,neutral,14.11.2015 13:21,"Reinsalu: lähiajal karmistatakse terrorivastaseid seadusi, äärmuslastel ei tohi olla mingit kohta oma mürgi külvamisel",EkspressGrupp,Delfi,NaN,['rahvus'],['islam'],NaN,"Text(text='See sõda tähendab, et peame ka Eestis oma julgeolekus arvesse võtma islami ekstremismi ohu hoopis uut olukorda silmas pidades.')",1,19.0,71.060000,8.0,['Eestis'],1.0
5,"”“Läbimõeldud, kogukonna juhitud ja täielikult ressurssidega kaetud tegevuskava, mis tegeleb mustanahaliste konkreetsete väljakutsetega, peab olema esmatähtis,” kirjutas Cullors ja lisas, et “me tahame oma häälte eest midagi.”BLM-i asutaja väitis koguni, et Biden ja Harris võitsid 2020. aasta presidendivalimised tänu mustanahalistele

In [ ]:
# morphological analysis by estNLTK
data["morph_analysis"] = data["sentence"].progress_apply(lambda s: Text(s).tag_layer())

100%|██████████| 3261/3261 [00:35<00:00, 90.68it/s] 


In [ ]:
# flesch score
sentence_flesch_score_retagger = SentenceFleschScoreRetagger()

# named entity tagging
ner_tagger = NerTagger()

# stopword list
url_stopwords = "https://raw.githubusercontent.com/kristel-/estonian-stopwords/master/estonian-stopwords.txt"
stopwords = pd.read_csv(url_stopwords, header=None, names=['stopword'])
stopword_list = list(stopwords.stopword)

In [ ]:
# feature engineering


def get_sentence_count(morph_layer):
  return len(morph_layer.sentences)


def get_word_count(morph_layer):
  return sum(1 for pos in morph_layer.partofspeech if "Z" not in pos) / len(morph_layer.sentences)


def get_dep_tree_height(morph_layer):
  try:
    morph_layer.tag_layer('maltparser_syntax')
    sentence_count = len(morph_layer.sentences)

    if sentence_count == 1:
      text_conll = conll_exporter.sentence_to_conll(morph_layer.sentences[0], morph_layer.maltparser_syntax)
      graph = DependencyGraph(text_conll, top_relation_label="root")
      dep_tree_height = graph.tree().height()

    elif sentence_count > 1:
      heights = []

      for sent in morph_layer.sentences:
        sent_conll = conll_exporter.sentence_to_conll(sent, morph_layer.maltparser_syntax)
        graph = DependencyGraph(sent_conll, top_relation_label="root")
        dep_tree_height = graph.tree().height()
        heights.append(dep_tree_height)

      dep_tree_height = np.average(heights)
      return dep_tree_height

  except:
    print(morph_layer.sentences.text)
    dep_tree_height = -1

  return dep_tree_height


def flesch_score(morph_layer):
  # 206.835 - 1.015 * (total_words / total_sentences) \ - 84.6 * (total_syllables / total_words)
  sentence_flesch_score_retagger.retag(morph_layer)
  return morph_layer.meta['whole_text_flesch_score']


def get_named_entities(morph_layer):
  ner_tagger.tag(morph_layer)
  # return morph_layer.ner.text
  return [named_entity.enclosing_text for named_entity in morph_layer.ner]


def get_noun_phrases(sentence):
  text = Text(sentence).tag_layer("np_chunks")
  noun_phrases = [phrase.text for phrase in text.np_chunks.spans if len(phrase.text) >= 2]
  return noun_phrases


def get_adjectives(morph_layer):
  return [adj.text.lower() for adj in morph_layer.words if adj.partofspeech[0] in ["A", "C", "U"]]


def get_quote_count(morph_layer):
  return sum(1 for q in morph_layer.words.text if q in ['"', '“', '”', '„', "‘", "‚", "«", "»"]) / len(morph_layer.sentences)

quotes = ['"', '“', '”', '„', "‘", "‚", "«", "»"]
puncts = [".", "!", "?", ","]
def get_quoted_words(sentence, lim=5):

  quoted_phrases = []
  quoted = re.split(r'["“”„‘‚«»]', sentence)

  # no quotes, return empty array
  if quoted[0] == sentence:
    return quoted_phrases

  for phrase in quoted:
    clean = phrase.strip()

    if phrase != clean or len(phrase) <= 2:
      continue
    if clean[0].isupper() and clean[-1] in puncts:
      continue
    if clean[0] in puncts:
      continue

    words = clean.split()
    if 1 <= len(words) <= lim:
      quoted_phrases += [clean]

  return quoted_phrases


def get_diminutives(morph_layer):
  return [word.lower() for root, word, pos in zip(morph_layer.root, morph_layer.words.text, morph_layer.partofspeech) \
          if ("=" in root[0] and (root[0].endswith("ke") or root[0].endswith("kene"))) \
          or (pos[0] in "ACU") \
          and ((root[0].endswith("ke") and word.lower().endswith("ke") and root[0] != word.lower()) or root[0].endswith("kene"))]


def get_superlatives(morph_layer):
  superlatives = []
  for i, adj in enumerate(morph_layer.words):
    if adj.partofspeech[0] == "U":
      superlatives += [adj.text.lower()]
    if adj.partofspeech[0] == "C" and morph_layer.words[i-1].text.lower() == "kõige":
      superlatives += ["kõige " + adj.text.lower()]
  return superlatives


def get_conditionals(morph_layer):
  return [word for form, word in zip(morph_layer.form, morph_layer.words.text) if "ks" in form[0] and "maks" not in form[0] and "takse" not in form[0]]


def get_translatives(morph_layer):
  return [word.lower() for form, word in zip(morph_layer.form, morph_layer.words.text) if "tr" in form[0]]


def get_indirects(morph_layer):
  return [word for form, word in zip(morph_layer.form, morph_layer.words.text) if "vat" in form[0]]


def get_stopwords(morph_layer):
  return [word.text.lower() for word in morph_layer.words if word.text.lower() in stopword_list]


def get_punct_count(morph_layer):
  return sum(1 for word in morph_layer.words if word.partofspeech[0] == "Z") / len(morph_layer.sentences)


bw_words = ["kõik", "kõige", "kunagi", "eales", "iial", "alati", "igavesti", "tervenisti", "täiesti", "üleni", "täitsa", "täielikult", "üdini", "läbinisti", "läbini", "absoluutne", "absoluutselt", "totaalne", "totaalselt", "ainult", "ainus", "kogu"]
def get_bw_count(morph_layer):
  return sum(1 for word in morph_layer.words if (word.text.lower() in bw_words or word.lemma[0] in bw_words)) / len(morph_layer.sentences)

In [ ]:
# extract all features

data["sentence_count"] = data["morph_analysis"].progress_apply(lambda ma: get_sentence_count(ma))

data["word_count"] = data["morph_analysis"].progress_apply(lambda ma: get_word_count(ma))
data["dependency_tree_height"] = data["morph_analysis"].progress_apply(lambda ma: get_dep_tree_height(ma))
data["flesch_score"] = data["morph_analysis"].progress_apply(lambda ma: flesch_score(ma))

data["named_entities"] = data["morph_analysis"].progress_apply(lambda ma: get_named_entities(ma))
data["named_entities_count"] = data.apply(lambda row: len(row["named_entities"]) / row["sentence_count"], axis=1)

100%|██████████| 3261/3261 [03:43<00:00, 14.59it/s]


In [ ]:
data["noun_phrases"] = data["sentence"].progress_apply(lambda s: get_noun_phrases(s))
data["noun_phrases_count"] = data.apply(lambda row: len(row["noun_phrases"]) / row["sentence_count"], axis=1)

data["morph_analysis"] = data["sentence"].progress_apply(lambda s: Text(s).tag_layer())

data["adjectives"] = data["morph_analysis"].progress_apply(lambda ma: get_adjectives(ma))
data["adjectives_count"] = data.apply(lambda row: len(row["adjectives"]) / row["sentence_count"], axis=1)

data["quotes_count"] = data["morph_analysis"].progress_apply(lambda ma: get_quote_count(ma))
data["quoted_words"] = data["sentence"].progress_apply(lambda s: get_quoted_words(s))
data["quoted_words_count"] = data.apply(lambda row: len(row["quoted_words"]) / row["sentence_count"], axis=1)

data["diminutives"] = data["morph_analysis"].progress_apply(lambda ma: get_diminutives(ma))
data["diminutives_count"] = data.apply(lambda row: len(row["diminutives"]) / row["sentence_count"], axis=1)

data["superlatives"] = data["morph_analysis"].progress_apply(lambda ma: get_superlatives(ma))
data["superlatives_count"] = data.apply(lambda row: len(row["superlatives"]) / row["sentence_count"], axis=1)

data["conditionals"] = data["morph_analysis"].progress_apply(lambda ma: get_conditionals(ma))
data["conditionals_count"] = data.apply(lambda row: len(row["conditionals"]) / row["sentence_count"], axis=1)

data["translatives"] = data["morph_analysis"].progress_apply(lambda ma: get_translatives(ma))
data["translatives_count"] = data.apply(lambda row: len(row["translatives"]) / row["sentence_count"], axis=1)

data["indirects"] = data["morph_analysis"].progress_apply(lambda ma: get_indirects(ma))
data["indirects_count"] = data.apply(lambda row: len(row["indirects"]) / row["sentence_count"], axis=1)

data["punctuations_count"] = data["morph_analysis"].progress_apply(lambda ma: get_punct_count(ma))
data["stopwords"] = data["morph_analysis"].progress_apply(lambda ma: get_stopwords(ma))
data["stopwords_count"] = data.apply(lambda row: len(row["stopwords"]) / row["sentence_count"], axis=1)
data["unique_stopwords"] = data["stopwords"].progress_apply(lambda sw: list(set(list(sw))))
data["unique_stopwords_count"] = data["unique_stopwords"].progress_apply(lambda x: len(x))

data["bw_count"] = data["morph_analysis"].progress_apply(lambda ma: get_bw_count(ma))

100%|██████████| 3261/3261 [00:00<00:00, 3586.90it/s]


In [ ]:
# download file (google drive)
from google.colab import drive
from google.colab import files

drive.mount("/content/drive")
data.to_csv("/content/drive/MyDrive/EstonianStanceDetection/notebooks/data.csv", index=False)
# files.download("data.csv")

In [ ]:
# local
data.to_csv("data.csv", index=False)